In [1]:
# 多種語系 本機範例
    # heroku遠端無法使用！
import os
from flask import Flask, redirect, render_template, url_for, request
import tempfile
from werkzeug.utils import secure_filename

from PIL import Image
import pytesseract

from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *

app = Flask(__name__)
# 上傳資料夾設定
app.config['UPLOAD_FOLDER'] = './static/'
# 本機__file__改成'__file__'
static_tmp_path = os.path.join(os.path.dirname('__file__'), 'static')

YOUR_CHANNEL_ACCESS_TOKEN = 'FEDPYiw8PpZJogWg4Kvq4qSDvQAAGKRSm7QXf3rzwzsCci/5e5BbyWIlSmessK/IdXz/brYCAFGDvroKU9tKukNOQ6z+gwMwrYFdTEQLB2waDBSr4OLU1LQL4fwPQMeXwmv7eXcBp6LVDc3tU0tfgAdB04t89/1O/w1cDnyilFU='
line_bot_api = LineBotApi(YOUR_CHANNEL_ACCESS_TOKEN)
YOUR_CHANNEL_SECRET = 'e812479abd804c2fd4e65bb99536d7e8'
handler = WebhookHandler(YOUR_CHANNEL_SECRET)

# 翻譯指定內容回傳首頁
@app.route('/')
def index():
    image = Image.open(r'static/photo.jpeg')
    code = pytesseract.image_to_string(image)
    print(code)
    return render_template('index.html', codes=code)

# line
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

# 使用者丟文字
dist_name = ''
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global dist_name
    msg = event.message.text
    if msg == '辨識範例':
        url1 = url2 = 'https://i.imgur.com/jvrlSjo.jpg'
        img = Image.open('static/sample.jpg')
        text = pytesseract.image_to_string(img, lang='eng')
        line_bot_api.reply_message(
            event.reply_token, [ImageSendMessage(original_content_url=url1, preview_image_url=url2), 
                                TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=text)])
    elif msg == '英文數字':
        if dist_name != '':
            image = Image.open(os.path.join('static', dist_name))
            dist_name = ''
            code = pytesseract.image_to_string(image, lang='eng')
            print(code)
            line_bot_api.reply_message(event.reply_token, 
                                       [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
        else:
            code = '沒有圖片 無法辨識'
            line_bot_api.reply_message(event.reply_token, 
                           [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
    elif msg == '繁體中文':
        if dist_name != '':
            image = Image.open(os.path.join('static', dist_name))
            dist_name = ''
            code = pytesseract.image_to_string(image, lang='chi_tra')
            print(code)
            line_bot_api.reply_message(event.reply_token, 
                                       [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
        else:
            code = '沒有圖片 無法辨識'
            line_bot_api.reply_message(event.reply_token, 
                           [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
    elif msg == '簡體中文':
        if dist_name != '':
            image = Image.open(os.path.join('static', dist_name))
            dist_name = ''
            code = pytesseract.image_to_string(image, lang='chi_sim')
            print(code)
            line_bot_api.reply_message(event.reply_token, 
                                       [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
        else:
            code = '沒有圖片 無法辨識'
            line_bot_api.reply_message(event.reply_token, 
                           [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
    elif msg == '英文數字+繁體中文':
        if dist_name != '':
            image = Image.open(os.path.join('static', dist_name))
            dist_name = ''
            code = pytesseract.image_to_string(image, lang='chi_tra+eng')
            print(code)
            line_bot_api.reply_message(event.reply_token, 
                                       [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
        else:
            code = '沒有圖片 無法辨識'
            line_bot_api.reply_message(event.reply_token, 
                           [TextSendMessage(text='-圖片轉文字為-'), TextSendMessage(text=code)])
    else:
        carousel_template_message = TemplateSendMessage(
            alt_text='Carousel template', 
            template=CarouselTemplate(
                columns=[CarouselColumn(thumbnail_image_url='https://i.imgur.com/pHX1jsZ.jpg', 
                                        title='歡迎加入寶寶 英文數字圖片 辨識', text='請上傳 英文數字圖片 以供辨識', 
                                        actions=[MessageAction(label='辨識範例', text='辨識範例', 
                                                               image_url='https://i.imgur.com/tqUlzuS.png')])]))
        line_bot_api.reply_message(event.reply_token, carousel_template_message)
    
# 使用者丟文字以外：Other Message Type
@handler.add(MessageEvent, message=(ImageMessage, VideoMessage, AudioMessage))
def handle_content_message(event):
    global dist_name
    if isinstance(event.message, ImageMessage):
        ext = 'jpg'
    elif isinstance(event.message, VideoMessage):
        ext = 'mp4'
    elif isinstance(event.message, AudioMessage):
        ext = 'm4a'
    else:
        return
    # 取得內容
    message_content = line_bot_api.get_message_content(event.message.id)
    with tempfile.NamedTemporaryFile(dir=static_tmp_path, prefix=ext + '-', delete=False) as tf:
        for chunk in message_content.iter_content():
            tf.write(chunk)
        tempfile_path = tf.name

    dist_path = tempfile_path + '.' + ext   # 路徑
    dist_name = os.path.basename(dist_path) # 檔名
    os.rename(tempfile_path, dist_path)
    # 圖片URL
    getImg = request.host_url + os.path.join('static', dist_name)
    
    text_message = TextSendMessage(
            text='轉成什麼語系(請到智慧型手機上點選選項)',
            quick_reply=QuickReply(
                items=[QuickReplyButton(action=MessageAction(label="英文數字", text="英文數字"), 
                                        image_url = 'https://i.imgur.com/CguUx8G.png'), 
                       QuickReplyButton(action=MessageAction(label="繁體中文", text="繁體中文"), 
                                        image_url = 'https://i.imgur.com/MTpggqp.png'), 
                       QuickReplyButton(action=MessageAction(label="簡體中文", text="簡體中文"), 
                                        image_url = 'https://i.imgur.com/XTop4ip.png'), 
                       QuickReplyButton(action=MessageAction(label="英文數字+繁體中文", text="英文數字+繁體中文"), 
                                        image_url = 'https://i.imgur.com/k3qdDW9.png'), ]))
    line_bot_api.reply_message(event.reply_token, text_message)

if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='127.0.0.1', port=port)  

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2020 14:14:09] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 14:14:15] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 14:14:25] "POST /callback HTTP/1.1" 200 -


BREAKING THE STATUE

i have always known

i just didn't understand
the inner conflictions
arresting our hands
gravitating close enough
expansive distamce between
i couldn't give you more
but i meant everything
when the day comes

you find your heart

wants something more

than a viece and a part
your life will change
like astatue set free

to walk among us

to created estiny

we didn't break any rules
we didn't make mistakes
making beauty in loving
making lovine for days

SHILOW


127.0.0.1 - - [26/May/2020 14:14:31] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 14:14:38] "POST /callback HTTP/1.1" 200 -


“二 把最好的自己。
留給最後那個最懂妳的好的人。
《昔自己,去更好的地方》一角子


127.0.0.1 - - [26/May/2020 14:14:44] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 14:14:51] "POST /callback HTTP/1.1" 200 -


二二
长相思
【清】纳兰性德
山一程，水一程。身向榆
关那畔行，夜深干帐灯。
风一更，雪一更。联碎乡
心梦不成，故园无此声。


127.0.0.1 - - [26/May/2020 14:14:55] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 14:15:01] "POST /callback HTTP/1.1" 200 -


這是一個中英文混合的 sentencez

相信這樣會比較 internationalz

更何況這可以用 computer 來辨識 textz
真的非常 good.。


127.0.0.1 - - [26/May/2020 14:15:08] "POST /callback HTTP/1.1" 200 -
